In [1]:
from qiskit import QuantumProgram
from IBMQuantumExperience import IBMQuantumExperience
import Qconfig
api = IBMQuantumExperience(Qconfig.APItoken, {'url':Qconfig.config["url"]})
from qiskit.backends import discover_remote_backends
remote_backends = discover_remote_backends(api)

Here we look at two jobs defined for a 16 qubit device.

In [4]:
num = 16
shots = 8192

One implements the $d=7$ repetition code, and the other implements the $d=8$ code. Though results are extracted for the full 16 qubit device, these jobs act non-trivially on only 13 and 15 qubits, respectively.

The jobs were both run on 30th April 2018. In this notebook, the data will be extracted using `get_jobs()`. The QASM for each job is shown, and the data is processed to give the fraction of samples that yeild the result `1` for each qubit.

If implemented perfectly, all qubits should output `0` in both cases. This can be verified using simulations for [d=7](https://quantumexperience.ng.bluemix.net/share/code/7bb5a04cd51f16f4b18db7c8331dd58d/execution/2bc40fd2b3041e1d497e82c915060ed7) and [d=8](https://quantumexperience.ng.bluemix.net/share/code/2468b7c25cde3145734d58069a89bc7f/execution/bc4153d5df2841b82476e6c99bc98b79 ).

In [8]:
jobs = api.get_jobs(limit=50)

id = {}
id[7] = "002c53c801d7c3fe9e42c11f130292a4"
id[8] = "e2baaa5dca050780c5144d61c8d0780c"


for job in jobs:
    
    for d in id.keys():
    
        if job['id']==id[d]:
        
            print( "\n\n\nImplementation of d="+str(d)+" repetition code.")
            print( "\nJob ID: " + job['id'] )
            print( "\nSubmitted at " + job['creationDate'] )
            print( "\nQASM:")
            print(job['qasms'][0]['qasm'])
            
            print("\nFraction of outcome 1 for each qubit\n")
            results = job['qasms'][0]['result']['data']['counts']
            p = [0]*num
            for qubit in range(num):
                for string in results.keys():
                    if string[num-1-qubit]=='1':
                        p[qubit] += results[string]/shots

                print( "Q" + str(qubit) + " "*(qubit<10) + "  " + " "*int(15*p[qubit]) + str(p[qubit]) )
        





Implementation of d=7 repetition code.

Job ID: 002c53c801d7c3fe9e42c11f130292a4

Submitted at 2018-04-30T22:07:50.564Z

QASM:

include "qelib1.inc";
qreg q[16];
creg c[16];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
cx q[15],q[14];
u2(0,3.14159265358979) q[13];
u2(0,3.14159265358979) q[12];
cx q[11],q[10];
cx q[5],q[4];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[2];
cx q[1],q[0];
cx q[12],q[13];
cx q[2],q[3];
u2(0,3.14159265358979) q[13];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[2];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
cx q[13],q[14];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[11];
cx q[9],q[10];
cx q[3],q[4];
cx q[1],q[2];
cx q[15],q[0];
cx q[12],q[11];
u2(0,3.14159265358979) q[12];
u2(0,3.14159265358979) q[11];
barrier q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7],q[8],q[9],q[10],q[11],q[12],q[13],q[14],q[15];
me

Note that the $d=8$ case deviates strongly from the ideal on qubits 0, 8, 11 and 12. These are mostly more likely to output `1` than `0`.

These features are not present for $d=7$, which typically output `1` 10-20% of the time. This is not perfect, but it is within expectations given the [reported values for noise](https://quantumexperience.ng.bluemix.net/qx/devices).